In [1]:
import sys
import string
import itertools
from collections import Counter, defaultdict
import re

from pathlib import Path
import os

import pandas as pd
import numpy as np

In [2]:
data = Path('../data/day_04.txt').read_text()

In [3]:
data = data.split('\n\n')

In [4]:
numbers, *boards = data

In [5]:
numbers = [int(k) for k in numbers.split(',')]

In [6]:
boards = [[[int(k) for k in row.split()] for row in board.splitlines()] for board in boards]

In [7]:
class Board:
    def __init__(self, board):
        self.pos_mapper = {v: (x, y) for (x, row) in enumerate(board) for (y, v) in enumerate(row)}
        self.column_bingos = Counter()
        self.row_bingos = Counter()
        self.board_size = len(board)
        self.column_max = 0
        self.row_max = 0
    
    def play_number(self, number):

        x, y = self.pos_mapper[number]
        self.column_bingos[y] += 1
        self.row_bingos[x] += 1
        self.column_max = max(self.column_max, self.column_bingos[y])
        self.row_max = max(self.row_max, self.row_bingos[x])
        
    @property
    def bingo(self):
        return self.column_max == self.board_size or self.row_max == self.board_size

In [8]:
%%timeit

boards_obj_list = dict(enumerate([Board(board) for board in boards]))
bingo_board = None

called = set()
last_called_number = None

for number in numbers:
    # print(number)
    called.add(number)
    for board_id, board in boards_obj_list.items():
        if number not in board.pos_mapper:
            continue
        board.play_number(number)
        if board.bingo:
            bingo_board = board
            last_called_number = number
            break
    if bingo_board:
        break

# print(number, bingo_board)
sum(bingo_board.pos_mapper.keys() - called) * number

1.53 ms ± 65 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit boards_obj_list = [Board(board) for board in boards] # re-initialization required

518 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%%timeit

boards_obj_list = [Board(board) for board in boards] # re-initialization required
board_mapping = dict(enumerate(boards_obj_list))
non_bingoed_boards = set(board_mapping)
last_bingo_board = None
called_numbers = set()

for number in numbers:
    # print("START", number, len(non_bingoed_boards))
    bingoed_boards = set()
    called_numbers.add(number)
    for board_id in non_bingoed_boards:
        board = board_mapping[board_id]
        if number not in board.pos_mapper:
            continue
        board.play_number(number)
        if board.bingo:
            # print(board_id)
            bingoed_boards.add(board_id)
            last_bingo_board = board
    non_bingoed_boards = non_bingoed_boards - bingoed_boards
    # print("END", number, len(non_bingoed_boards))
    if not non_bingoed_boards:
        # print(number, 'break')
        break
    # print()
# print(number, last_bingo_board)
sum(last_bingo_board.pos_mapper.keys() - called_numbers) * number

3.32 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
